In [11]:
# ==========================================
# NOTEBOOK 06: AI PREDICTIVE CONTROL APP (INTERACTIVE)
# ==========================================
import time
import pandas as pd
import numpy as np
import joblib

print("[SYSTEM] Loading AI Models and Historical Environment Data...")

# 1. LOAD DATA & MODELS (For validation ranges)
df_batter = pd.read_csv('../data/Muestreo_Depositado_Presion.csv')
df_cream = pd.read_csv('../data/Muestreo_Crema_Presion_V2.csv')
df_jam = pd.read_csv('../data/Muestreo_Mermelada_Presion.csv')

df_batter.rename(columns={'Peso (g)': 'Weight_g', 'Temperatura (°C)': 'Temp_C', 'Viscosidad (mPa.s)': 'Viscosity_mPas', '% Reproceso': 'Rework_pct'}, inplace=True)
df_cream.rename(columns={'Peso (g)': 'Weight_g', 'Temperatura (°C)': 'Temp_C', 'Viscosidad (mPa.s)': 'Viscosity_mPas'}, inplace=True)
df_jam.rename(columns={'Peso (g)': 'Weight_g', 'Temperatura (°C)': 'Temp_C', 'Viscosidad (mPa.s)': 'Viscosity_mPas'}, inplace=True)

datasets = {'Batter': {'data': df_batter}, 'Cream': {'data': df_cream}, 'Jam': {'data': df_jam}}

models = {
    'Batter': joblib.load('../models/optimizer_batter.pkl'),
    'Cream': joblib.load('../models/optimizer_cream.pkl'),
    'Jam': joblib.load('../models/optimizer_jam.pkl')
}

# 2. CORE FUNCTIONS
def calculate_exact_pressure(stage_name, temp, viscosity, target_weight, rework=0.0):
    model = models[stage_name]
    pressures_to_test = np.linspace(2.0, 9.0, 100)
    results = []
    
    calibration_offset = 4.8 if stage_name == 'Batter' else 1.5
    
    for p in pressures_to_test:
        if stage_name == 'Batter':
            input_df = pd.DataFrame([[temp, viscosity, rework, p]], columns=['Temp_C', 'Viscosity_mPas', 'Rework_pct', 'Pressure_bar'])
        else:
            input_df = pd.DataFrame([[temp, viscosity, p]], columns=['Temp_C', 'Viscosity_mPas', 'Pressure_bar'])
            
        pred_weight_historical = model.predict(input_df)[0]
        pred_weight_calibrated = pred_weight_historical - calibration_offset
        
        error = abs(pred_weight_calibrated - target_weight)
        results.append((p, error, pred_weight_calibrated))
    
    best_match = min(results, key=lambda x: x[1])
    return round(best_match[0], 2), round(best_match[2], 2)

def get_validated_input(prompt_text, min_val, max_val):
    while True:
        try:
            # Pide el dato al usuario
            user_input = input(f"{prompt_text} [Allowed: {min_val:.1f} to {max_val:.1f}]: ")
            value = float(user_input)
            if min_val <= value <= max_val:
                return value
            else:
                print(f"  [ERROR] Value must be between {min_val:.1f} and {max_val:.1f}.")
        except ValueError:
            print("  [ERROR] Please enter a valid numerical value.")

# 3. INTERACTIVE TERMINAL
print("\n" + "="*65)
print(" AI PREDICTIVE CONTROL SYSTEM ")
print("="*65)

print("\nAvailable Production Lines:")
print("  [1] Batter (Bread)")
print("  [2] Cream")
print("  [3] Jam")

opcion = input("\n[INPUT] Select the stage to optimize (1-3): ")

stage_map = {'1': 'Batter', '2': 'Cream', '3': 'Jam'}
name_map = {'1': 'BATTER (BREAD)', '2': 'CREAM', '3': 'JAM'}

if opcion not in stage_map:
    print("\n[ERROR] Invalid option. Please run the cell again to retry.")
else:
    stage_name = stage_map[opcion]
    df_current = datasets[stage_name]['data'] 
    
    min_t, max_t = df_current['Temp_C'].min(), df_current['Temp_C'].max()
    min_v, max_v = df_current['Viscosity_mPas'].min(), df_current['Viscosity_mPas'].max()
    
    print(f"\n--- ENTERING PARAMETERS FOR: {name_map[opcion]} ---")
    
    target_w = get_validated_input("[INPUT] What is the TARGET DEPOSIT WEIGHT? (g)", 3.0, 30.0)
    temp = get_validated_input("[INPUT] Enter current fluid Temperature (°C)", min_t, max_t)
    visc = get_validated_input("[INPUT] Enter current Viscosity (mPa.s)", min_v, max_v)
    
    rework = 0.0
    if stage_name == 'Batter':
        min_r, max_r = df_current['Rework_pct'].min(), df_current['Rework_pct'].max()
        rework = get_validated_input("[INPUT] Enter mixture Rework %", min_r, max_r)
        
    print("\n[SYSTEM] Calculating thermodynamics and finding optimal pressure...")
    time.sleep(1) 
    
    p_opt, w_exp = calculate_exact_pressure(stage_name, temp, visc, target_w, rework)
    
    print("\n" + "*"*55)
    print(f" REQUESTED TARGET WEIGHT : {target_w} g")
    print(f" EXACT REQUIRED PRESSURE : {p_opt} bar")
    print(f" AI PROJECTED WEIGHT     : {w_exp} g")
    print("*"*55)

[SYSTEM] Loading AI Models and Historical Environment Data...

 AI PREDICTIVE CONTROL SYSTEM 

Available Production Lines:
  [1] Batter (Bread)
  [2] Cream
  [3] Jam

--- ENTERING PARAMETERS FOR: BATTER (BREAD) ---

[SYSTEM] Calculating thermodynamics and finding optimal pressure...

*******************************************************
 REQUESTED TARGET WEIGHT : 19.9586 g
 EXACT REQUIRED PRESSURE : 4.76 bar
 AI PROJECTED WEIGHT     : 15.41 g
*******************************************************
